In [1]:
%load_ext sql

In [2]:
%sql sqlite://

# Structured Query Language (SQL)

## Where Were We?

1. Language primitives (i.e., building blocks of languages)
2. **Language paradigms** (i.e., combinations of language primitives)
    - Last time: DSL with regular expressions
    - This time: DSL for data management called **SQL** (structured query language)
3. Building a language (i.e., designing your own language)

## Background

SQL is:

* **Declarative**: You don't tell it _how_ to do things, only _what you want_.
* **Domain-specific**: Specialized for data query and manipulation.

SQL is used to access data in a **relational database**:

* Relational databases are made up of **tables** (relations). Think **spreadsheet**.
* All data is in tables: both the raw data and the output of queries.
* Based on **relational algebra** by Edgar F. Codd (1970). (https://en.wikipedia.org/wiki/Relational_algebra)


# Sample application: Shopping List App

![Items](images/items.png)

# Creating a table of items

Note how SQL reads a bit like English. **It was aimed at business people**, not programmers.

In [3]:
%%sql

DROP TABLE IF EXISTS item;

CREATE TABLE item(
    item_id INTEGER PRIMARY KEY,   -- unique numberical ID for item
    value TEXT NOT NULL,           -- name of item, like 'milk'
    category TEXT NOT NULL)        -- section of the store, like 'dairy'

 * sqlite://
Done.
Done.


[]

```
item
---------------------------------------------------
| item_id: INTEGER | value: TEXT | category: TEXT |
---------------------------------------------------
```

# Inserting data

In [4]:
%%sql

INSERT INTO item VALUES
    (1, 'milk', 'dairy'),
    (2, 'bananas', 'produce'),
    (3, 'eggs', 'dairy'),
    (4, 'bread', 'bakery'),
    (5, 'apples', 'produce'),
    (6, 'onions', 'produce');


 * sqlite://
6 rows affected.


[]

```
item
---------------------------------------------------
| item_id: INTEGER | value: TEXT | category: TEXT |
---------------------------------------------------
|       1          |   milk      | dairy          |
---------------------------------------------------
|       2          |   banans    | produce        |
---------------------------------------------------
....
```

# Querying data

In [5]:
%%sql

SELECT *
FROM item;          -- It's customary to wrap queries at each clause.

 * sqlite://
Done.


item_id,value,category
1,milk,dairy
2,bananas,produce
3,eggs,dairy
4,bread,bakery
5,apples,produce
6,onions,produce


Note that rows are **inherently unordered**. Can ask for just a single column:

In [6]:
%%sql

SELECT value
FROM item;

 * sqlite://
Done.


value
milk
bananas
eggs
bread
apples
onions


Or multiple columns:

In [7]:
%%sql

SELECT value, category
FROM item;

 * sqlite://
Done.


value,category
milk,dairy
bananas,produce
eggs,dairy
bread,bakery
apples,produce
onions,produce


Or arbitrary expressions:

In [8]:
%%sql

SELECT item_id*10
FROM item;

 * sqlite://
Done.


item_id*10
10
20
30
40
50
60


**Question**: Does this remind you of a functional construct we've studied?

# WHERE clause

In [9]:
%%sql

SELECT *
FROM item
WHERE item_id = 4;

 * sqlite://
Done.


item_id,value,category
4,bread,bakery


In [10]:
%%sql

SELECT *
FROM item
WHERE item_id >= 4 OR category = 'produce';

 * sqlite://
Done.


item_id,value,category
2,bananas,produce
4,bread,bakery
5,apples,produce
6,onions,produce


**Question**: Does this remind you of a functional construct we've studied?

Advantage of declarative: database can optimize `WHERE` clause with **indexes**.

# ORDER BY clause

In [11]:
%%sql

SELECT *
FROM item
ORDER BY category, value;

 * sqlite://
Done.


item_id,value,category
4,bread,bakery
3,eggs,dairy
1,milk,dairy
5,apples,produce
2,bananas,produce
6,onions,produce


# Aggregate functions

In [12]:
%%sql

SELECT count(item_id)
FROM item;

 * sqlite://
Done.


count(item_id)
6


In [13]:
%%sql

SELECT count(*)
FROM item
WHERE category = 'dairy';

 * sqlite://
Done.


count(*)
2


**Question**: Does `count` remind you of a functional construct we've studied?

# GROUP BY clause

In [14]:
%%sql

SELECT category, count(*)
FROM item
GROUP BY category;

 * sqlite://
Done.


category,count(*)
bakery,1
dairy,2
produce,3


# Multiple tables

We want to multiple shopping lists, but also recipes. We only store the _items_ of recipes, not the instructions.

![lists](images/lists.png)


In [15]:
%%sql

DROP TABLE IF EXISTS list;

CREATE TABLE list(
    list_id INTEGER PRIMARY KEY,    -- unique key for list
    list_name TEXT NOT NULL,        -- name of list
    list_type TEXT NOT NULL)        -- 'shopping' or 'recipe'

 * sqlite://
Done.
Done.


[]

In [16]:
%%sql

INSERT INTO list VALUES
    (101, 'Safeway', 'shopping'),
    (102, 'Costco', 'shopping'),
    (103, 'Caesar salad', 'recipe'),
    (104, 'Meatloaf', 'recipe');
    

 * sqlite://
4 rows affected.


[]

In [17]:
%%sql

SELECT *
FROM list;

 * sqlite://
Done.


list_id,list_name,list_type
101,Safeway,shopping
102,Costco,shopping
103,Caesar salad,recipe
104,Meatloaf,recipe


# ALTER TABLE command

Each item needs to know which list it's in.

Note that the lists are not containers, like arrays, that we put items into. Lists and items are in separate tables, linked by the list ID. This is a key way that relational databases are organized.

In [18]:
%%sql

ALTER TABLE item
    ADD COLUMN list_id INTEGER;

 * sqlite://
Done.


[]

But each item is not yet in any list:

In [19]:
%%sql

SELECT *
FROM item;

 * sqlite://
Done.


item_id,value,category,list_id
1,milk,dairy,None
2,bananas,produce,None
3,eggs,dairy,None
4,bread,bakery,None
5,apples,produce,None
6,onions,produce,None


# UPDATE command

In [20]:
%%sql

UPDATE item
    SET list_id = 101;

 * sqlite://
6 rows affected.


[]

In [21]:
%%sql

SELECT *
FROM item;

 * sqlite://
Done.


item_id,value,category,list_id
1,milk,dairy,101
2,bananas,produce,101
3,eggs,dairy,101
4,bread,bakery,101
5,apples,produce,101
6,onions,produce,101


In [22]:
%%sql

INSERT INTO item VALUES

    -- Costco:
    (7, 'paper towels', 'household', 102),
    (8, 'cereal', 'breakfast', 102),
    (9, 'soda', 'drinks', 102),
    
    -- Caesar salad recipe:
    (10, 'romaine lettuce', 'produce', 103),
    (11, 'caesar dressing', 'baking', 103),
    (12, 'croutons', 'bakery', 103),
    
    -- Meatloaf recipe:
    (13, 'ground beef', 'meat', 104),
    (14, 'onions', 'produce', 104),          -- duplicate of onions on Safeway list, that's okay
    (15, 'bread crumbs', 'bakery', 104),
    (16, 'eggs', 'dairy', 104);
    

 * sqlite://
10 rows affected.


[]

In [23]:
%%sql

SELECT *
FROM item;

 * sqlite://
Done.


item_id,value,category,list_id
1,milk,dairy,101
2,bananas,produce,101
3,eggs,dairy,101
4,bread,bakery,101
5,apples,produce,101
6,onions,produce,101
7,paper towels,household,102
8,cereal,breakfast,102
9,soda,drinks,102
10,romaine lettuce,produce,103


Items in Costco shopping list:

![costco](images/costco.png)

In [24]:
%%sql

SELECT *
FROM list;

 * sqlite://
Done.


list_id,list_name,list_type
101,Safeway,shopping
102,Costco,shopping
103,Caesar salad,recipe
104,Meatloaf,recipe


In [25]:
%%sql

SELECT list_id
FROM list
WHERE list_name = 'Costco';

 * sqlite://
Done.


list_id
102


In [26]:
%%sql

SELECT value, category
FROM item
WHERE list_id = 103;

 * sqlite://
Done.


value,category
romaine lettuce,produce
caesar dressing,baking
croutons,bakery


# Joins

Can specify multiple lists. Get the **cross product**: every combination of rows from each table.

In [27]:
%%sql

SELECT *
FROM item, list;

 * sqlite://
Done.


item_id,value,category,list_id,list_id_1,list_name,list_type
1,milk,dairy,101,101,Safeway,shopping
1,milk,dairy,101,102,Costco,shopping
1,milk,dairy,101,103,Caesar salad,recipe
1,milk,dairy,101,104,Meatloaf,recipe
2,bananas,produce,101,101,Safeway,shopping
2,bananas,produce,101,102,Costco,shopping
2,bananas,produce,101,103,Caesar salad,recipe
2,bananas,produce,101,104,Meatloaf,recipe
3,eggs,dairy,101,101,Safeway,shopping
3,eggs,dairy,101,102,Costco,shopping


Can keep only rows where the list IDs match:

In [28]:
%%sql

SELECT *
FROM item, list
WHERE item.list_id = list.list_id;

 * sqlite://
Done.


item_id,value,category,list_id,list_id_1,list_name,list_type
1,milk,dairy,101,101,Safeway,shopping
2,bananas,produce,101,101,Safeway,shopping
3,eggs,dairy,101,101,Safeway,shopping
4,bread,bakery,101,101,Safeway,shopping
5,apples,produce,101,101,Safeway,shopping
6,onions,produce,101,101,Safeway,shopping
7,paper towels,household,102,102,Costco,shopping
8,cereal,breakfast,102,102,Costco,shopping
9,soda,drinks,102,102,Costco,shopping
10,romaine lettuce,produce,103,103,Caesar salad,recipe


Get all contents of shopping lists:

In [29]:
%%sql

SELECT value, category, list_name
FROM item, list
WHERE item.list_id = list.list_id
AND list_type = 'shopping';

 * sqlite://
Done.


value,category,list_name
milk,dairy,Safeway
bananas,produce,Safeway
eggs,dairy,Safeway
bread,bakery,Safeway
apples,produce,Safeway
onions,produce,Safeway
paper towels,household,Costco
cereal,breakfast,Costco
soda,drinks,Costco


Get list summary:

![lists with counts](images/lists-with-counts.png)

In [30]:
%%sql

SELECT list_name, count(*)
FROM item, list
WHERE item.list_id = list.list_id
GROUP BY list_name;

 * sqlite://
Done.


list_name,count(*)
Caesar salad,3
Costco,3
Meatloaf,4
Safeway,6


Get all items in Safeway shopping list:

In [31]:
%%sql

SELECT value, category
FROM item
WHERE list_id = 101;

 * sqlite://
Done.


value,category
milk,dairy
bananas,produce
eggs,dairy
bread,bakery
apples,produce
onions,produce


Easier: Use join to specify list by name instead of ID. Also sort by category:

![safeway](images/safeway.png)

In [32]:
%%sql

SELECT value, category
FROM item, list
WHERE item.list_id = list.list_id
AND list_name = 'Safeway'
ORDER BY category;

 * sqlite://
Done.


value,category
bread,bakery
milk,dairy
eggs,dairy
bananas,produce
apples,produce
onions,produce


# Query plans and indexes

In [33]:
%%sql

DROP INDEX IF EXISTS list_list_name;
DROP INDEX IF EXISTS item_list_id;

EXPLAIN QUERY PLAN
SELECT value, category
FROM item, list
WHERE item.list_id = list.list_id
AND list_name = 'Safeway'
ORDER BY category;

 * sqlite://
Done.
Done.
Done.


id,parent,notused,detail
4,0,0,SCAN TABLE item
6,0,0,SEARCH TABLE list USING INTEGER PRIMARY KEY (rowid=?)
16,0,0,USE TEMP B-TREE FOR ORDER BY


In [34]:
%%sql

CREATE INDEX list_list_name ON list(list_name);
CREATE INDEX item_list_id ON item(list_id);

 * sqlite://
Done.
Done.


[]

In [35]:
%%sql

EXPLAIN QUERY PLAN
SELECT value, category
FROM item, list
WHERE item.list_id = list.list_id
AND list_name = 'Safeway'
ORDER BY category;

 * sqlite://
Done.


id,parent,notused,detail
5,0,0,SEARCH TABLE list USING COVERING INDEX list_list_name (list_name=?)
9,0,0,SEARCH TABLE item USING INDEX item_list_id (list_id=?)
20,0,0,USE TEMP B-TREE FOR ORDER BY


# Normalization

* **Question**: What if we want to rename a category?
* Our data is **unnormalized**.
* https://en.wikipedia.org/wiki/Database_normalization

# Complex queries.

SQL queries can get complicated. This one finds appropriate users to send an email to for a survey.

```sql

INSERT INTO tmp_mailing_list
SELECT
    person_id,
    email_address,
    lifetime_bucket,
    CASE
        WHEN ( random_ordinal_within_bucket > 1500 )
            THEN ( NULL )
        WHEN ( random_ordinal_within_bucket % 2 = 0 )
            THEN ( SELECT id FROM mailing WHERE description = 'Survey Monkey July 2017' )
        ELSE ( SELECT id FROM mailing WHERE description = 'Email survey July 2017' )
    END AS mailing_id,
    random_ordinal_within_bucket,
    lifetime,
    last_use
FROM (
    SELECT
        person_id,
        email_address,
        lifetime_bucket,
        row_number() OVER ( PARTITION BY lifetime_bucket ORDER BY random_number ) AS random_ordinal_within_bucket,
        lifetime,
        last_use
    FROM (
        SELECT
            person_ex.id AS person_id,
            email_address,
            CASE
                WHEN ( last_use - first_use < '2 days'::interval )
                    THEN '1: lt 2 days'
                WHEN ( last_use - first_use < '30 days'::interval )
                    THEN '2: lt 30 days'
                WHEN ( last_use - first_use < '6 months'::interval )
                    THEN '3: lt 6 months'
                ELSE '4: gte 6 months'
            END AS lifetime_bucket,
            last_use - first_use AS lifetime,
            last_use,
            RANDOM() AS random_number
        FROM
            ( SELECT
                    person.id,
                    person.email_address,
                    MIN( client.created_at ) AS first_use,
                    MAX( client.last_described_at ) AS last_use
            FROM client, person
            WHERE client.owning_person_id = person.id
              AND client.created_at IS NOT NULL
              AND person.email_address IS NOT NULL
              AND person.opt_out = FALSE
              AND person.email_address NOT LIKE '%,%'
            GROUP BY person.id ) AS person_ex -- all persons with email addresses, and the earliest and latest use
        WHERE last_use >= NOW() - '60 days'::interval
          AND last_use < NOW() - '30 days'::interval
    ) AS address_ex
) AS bucketed_addresses;

```

# Class project

Example from the class project:

```sql
CREATE TABLE songs (
        id int NOT NULL PRIMARY KEY,
        song_title TEXT NOT NULL,
        notes varchar NOT NULL
);

INSERT INTO songs (id, song_title, notes)
    VALUES (1, 'Ode to Joy (Dubstep Remix)', 'E4 E4 F4 G4 G4 F4 E4 D4 C4 C4 D4 E4 E4 D4 D4');
```

Creates the table of songs and inserts one song into it (you heard it in class).

# SQL injection

Here's how to access the database from programs:

```java
database.update("INSERT INTO list VALUES (?, ?, ?)", listId, listName, listType);
```

Note the `?` in the query. These are replaced by the parameters. This is the safe way to do it.

But there's another, unsafe way. **Do not do this**:

```java
database.update("INSERT INTO list VALUES (" + listId + ", '" + listName + "', '" + listType + "')");
```

What if the user tries to create a list with the name:

```
    safeway', 'shopping'); DROP TABLE list; --
```

The SQL you run will be:

```sql
INSERT INTO list VALUES (101, 'safeway', 'shopping'); DROP TABLE list; -- ', 'shopping')
```

![Exploits](images/xkcd_exploits.png)

# Type of relational databases

Commercial:

* Oracle.
* SQL Server.

Free:

* MySQL.
* PostgreSQL.

Embedded:

* SQLite. (Used in this class)

All use SQL.


# Summary

SQL is:

* **Declarative**: You don't tell it _how_ to do things, only _what you want_.
* **Domain-specific**: Specialized for data query and manipulation.
